In [2]:
#List of 10 state bus routes
list=['https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile', 'https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile','https://www.redbus.in/online-booking/astc/?utm_source=rtchometile','https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu','https://www.redbus.in/online-booking/kaac-transport','https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile','https://www.redbus.in/online-booking/jksrtc','https://www.redbus.in/online-booking/north-bengal-state-transport-corporation','https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile','https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile']
print(list)

['https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile', 'https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile', 'https://www.redbus.in/online-booking/astc/?utm_source=rtchometile', 'https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu', 'https://www.redbus.in/online-booking/kaac-transport', 'https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile', 'https://www.redbus.in/online-booking/jksrtc', 'https://www.redbus.in/online-booking/north-bengal-state-transport-corporation', 'https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile', 'https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile']


In [ ]:
#Extract Route data
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

driver = webdriver.Chrome()

def extract_data():
    hrefs = []
    texts = []
    scrolling = True
    while scrolling:
        route_data = driver.find_elements(By.XPATH, "//a[@class='route']")
        
        for route in route_data:
            hrefs.append(route.get_attribute('href'))
            texts.append(route.text)
        
        old_page_source = driver.page_source
        body = driver.find_element(By.TAG_NAME, "body")
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False
    
    return hrefs, texts
def scrape_url(url):
    driver.get(url)
    driver.maximize_window()
    time.sleep(5)
    
    all_hrefs, all_texts = extract_data()
    
    for page_number in range(2, 8):
        try:
            page_button = driver.find_element(By.XPATH, f"//div[@class='DC_117_pageTabs ' and normalize-space(text())='{page_number}']")
            driver.execute_script("arguments[0].scrollIntoView(true);", page_button)
            driver.execute_script("arguments[0].click();", page_button)
            time.sleep(5)
            
            page_hrefs, page_texts = extract_data()
            
            all_hrefs.extend(page_hrefs)
            all_texts.extend(page_texts)
            
        except Exception as e:
            print(f"Error navigating to page {page_number} or extracting data:", e)
            break
    
    return all_hrefs, all_texts

all_hrefs = []
all_texts = []

for url in list:
    try:
        hrefs, texts = scrape_url(url)
        all_hrefs.extend(hrefs)
        all_texts.extend(texts)
        
        print(f"URL: {url}")
        print("Hrefs:", hrefs)
        print("Texts:", texts)
        
    except Exception as e:
        print(f"Error scraping URL {url}:", e)

print("All Hrefs:", all_hrefs)
print("All Texts:", all_texts)

driver.quit()

In [4]:
#dataframe for route
import pandas as pd
Route_with_link=pd.DataFrame({'Route':all_texts,
                              'Route_link':all_hrefs})

In [5]:
Route_with_link

,Route,Route_link
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
1,Goa to Pune,https://www.redbus.in/bus-tickets/goa-to-pune
2,Mumbai to Goa,https://www.redbus.in/bus-tickets/mumbai-to-goa
3,Goa to Mumbai,https://www.redbus.in/bus-tickets/goa-to-mumbai
4,Pandharpur to Goa,https://www.redbus.in/bus-tickets/pandharpur-t...
...,...,...
482,Delhi to Nalagarh,https://www.redbus.in/bus-tickets/delhi-to-nal...
483,Baddi (Himachal Pradesh) to Delhi,https://www.redbus.in/bus-tickets/baddi-himach...
484,Kangra to Delhi,https://www.redbus.in/bus-tickets/kangra-to-delhi
485,Ghumarwin to Delhi,https://www.redbus.in/bus-tickets/ghumarwin-to...


In [6]:

import mysql.connector

def connect_to_mysql():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password=""
    )

connection = connect_to_mysql()

mycursor = connection.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS RedBus")
mycursor.execute("USE RedBus")

mycursor.execute('''CREATE TABLE IF NOT EXISTS Route_details (
                 Route VARCHAR(200),
                 Route_Link VARCHAR(200)
             )''')

sql = "INSERT INTO Route_details (Route, Route_Link) VALUES (%s, %s)"

for route, link in zip(all_texts, all_hrefs):
    val = (route, link)
    mycursor.execute(sql, val)

connection.commit()

mycursor.close()
connection.close()

print("Data inserted successfully")

Data inserted successfully


In [7]:
import pandas as pd
import mysql.connector

def connect_to_mysql():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password=""
    )

connection = connect_to_mysql()

query1 = "SELECT * FROM RedBus.Route_details  "
mycursor = connection.cursor(buffered=True)
mycursor.execute(query1)    
t1 = mycursor.fetchall()
Route_detail = pd.DataFrame(t1, columns=["Route","Route_Link"])


In [ ]:
Route_detail['Route_Link']

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

driver = webdriver.Chrome()
def scroll_and_load():
    max_scroll_attempts = 5  
    scroll_attempt = 0
    previous_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(3) 
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == previous_height:
            scroll_attempt += 1
            if scroll_attempt >= max_scroll_attempts:
                break
        else:
            scroll_attempt = 0 
        previous_height = new_height

def check_and_click_view_button():
    try:
        view_buses_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        view_buses_button.click()
        return True
    except:
        print("")
        return False

route_link = []
route = []
bus_type = []
departing_time = []
duration = []
reaching_time = []
star_rating = []
price = []
seat_available = []
bus_names = []
before_offer=[]

for url in  Route_detail['Route_Link']:
    k = 1  
    route_value = url[34:]  
    
    try:
        driver.get(url)
        driver.maximize_window()

        if check_and_click_view_button():
            scroll_and_load()
            time.sleep(5)
        
        scroll_and_load()

        ID_Andhra = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type_data = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_data = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        duration_data = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        reaching_data = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        price_data = driver.find_elements(By.XPATH, "//div[@class='fare d-block']")
        seat_available_data = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']")
        before_offer_data= driver.find_elements(By.XPATH, "//span[@class='strike']")
        for i in range(len(ID_Andhra)):
            star_rating_xpath = f"/html/body/section/div[2]/div[4]/div/div[2]/div/div[2]/div[2]/div/ul/div[{k}]/li/div/div[1]/div[1]/div[5]/div[1]/div/span"
            star_rating_data = driver.find_elements(By.XPATH, star_rating_xpath)
            bus_name = ID_Andhra[i].text
            bus_names.append(bus_name)
            route_link.append(url)
            route.append(route_value)
            bus_type.append(bus_type_data[i].text if i < len(bus_type_data) else "")
            departing_time.append(departing_data[i].text if i < len(departing_data) else "")
            duration.append(duration_data[i].text if i < len(duration_data) else "")
            reaching_time.append(reaching_data[i].text if i < len(reaching_data) else "")
            star_rating.append(star_rating_data[0].text if star_rating_data else "")
            print(star_rating)
            price.append(price_data[i].text if i < len(price_data) else"")
            before_offer.append(before_offer_data[i].text if i < len(before_offer_data) else"")
            seat_available.append(seat_available_data[i].text if i < len(seat_available_data) else "")
            k += 1  
    
    except Exception as e:
        print(f"Error processing URL '{url}': {str(e)}")

driver.quit()

In [10]:
df = pd.DataFrame({
    'Bus_name':bus_names,
    'Route Link': route_link,
    'Route': route,
    'Bus Type': bus_type,
    'Departing Time': departing_time,
    'Duration': duration,
    'Reaching Time': reaching_time,
    'Star Rating': star_rating,
    'Price': price,
    'Before_Price':before_offer,
    'Seat Available': seat_available,
})


In [24]:
df

,Bus_name,Route Link,Route,Bus Type,Departing Time,Duration,Reaching Time,Star Rating,Price,Before_Price,Seat Available
0,Kadamba Transport Corporation Limited (KTCL) -...,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,A/C Sleeper (2+1),19:15,10h 35m,05:50,3.6,INR 800,555,17 Seats available
1,Atmaram Gobus,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,VE A/C Sleeper (2+1),22:30,10h 15m,08:45,4.7,INR 565,576,13 Seats available
2,Zingbus Plus,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,Bharat Benz A/C Sleeper (2+1),21:45,10h 50m,08:35,4.5,486,586,24 Seats available
3,Atmaram Gobus,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,VE A/C Sleeper (2+1),21:00,10h 30m,07:30,4.5,INR 555,650,7 Seats available
4,Ashray Vip Yellows,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,A/C Sleeper (2+1),20:30,11h 30m,08:00,4.4,INR 820,600,19 Seats available
...,...,...,...,...,...,...,...,...,...,...,...
6956,HRTC - 78,https://www.redbus.in/bus-tickets/ghumarwin-to...,ghumarwin-to-delhi,Himsuta AC Seater Volvo/Scania 2+2,21:15,07h 45m,05:00,4.8,INR 892,,34 Seats available
6957,HRTC - 541,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,Ordinary 3+2 Non AC Seater,19:14,08h 56m,04:10,5.0,INR 575,,24 Seats available
6958,HRTC - 549,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,A/C Executive (2+3),19:25,11h 20m,06:45,2.7,INR 701,,37 Seats available
6959,HRTC - 557,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,Ordinary 3+2 Non AC Seater,20:15,10h 55m,07:10,3.8,INR 575,,28 Seats available


In [ ]:
import mysql.connector
from datetime import datetime

def connect_to_mysql():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password=""
    )

connection = connect_to_mysql()

mycursor = connection.cursor()

mycursor.execute("CREATE DATABASE IF NOT EXISTS RedBus")
mycursor.execute("USE RedBus")

mycursor.execute('''
    CREATE TABLE IF NOT EXISTS Bus_Details (
        Bus_name VARCHAR(200),
        Route_Link VARCHAR(200),
        Route VARCHAR(200),
        Bus_Type VARCHAR(200),
        Departing_Time TEXT,
        Duration Varchar(255),
        Reaching_Time TEXT,
        Star_Rating FLOAT,
        Price TEXT,
        Before_price float,
        Seat_Available INT
    )
''')



sql = '''
    INSERT INTO Bus_Details 
    (Bus_name, Route_Link, Route, Bus_Type, Departing_Time, Duration, Reaching_Time, Star_Rating, Price,Before_price, Seat_Available) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s)
'''

val = zip(bus_names, route_link, route, bus_type, departing_time, duration, reaching_time, star_rating, price, before_offer, seat_available)

for row in val:
    mycursor.execute(sql, row)

connection.commit() 

mycursor.close()
connection.close()

print("Data inserted successfully.")

In [12]:
import mysql.connector
from datetime import datetime

def connect_to_mysql():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password=""
    )

connection = connect_to_mysql()

mycursor = connection.cursor()
query = "SELECT Bus_name, Route_Link, Route, Bus_Type, Departing_Time, Duration, Reaching_Time, Star_Rating, Price, Seat_Available FROM RedBus.Bus_Details"
cursor = connection.cursor(buffered=True)
mycursor.execute(query)



In [13]:
data = mycursor.fetchall()

In [ ]:
data

In [15]:
columns = ["Bus_name", "Route_Link", "Route", "Bus_Type", "Departing_Time", "Duration", "Reaching_Time", "Star_Rating", "Price", "Seat_Available"]
Bus_Data = pd.DataFrame(data, columns=columns)

In [16]:
Bus_Data

,Bus_name,Route_Link,Route,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Available
0,Kadamba Transport Corporation Limited (KTCL) -...,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,Volvo AC Seater 2+2,20:30,09h 51m,06:21,4.0,INR 800,15
1,Atmaram Gobus,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,VE A/C Sleeper (2+1),21:00,10h 30m,07:30,4.5,INR 799,17
2,Ashray Travels,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,Bharat Benz A/C Sleeper (2+1),21:00,11h 30m,08:30,4.5,INR 849,20
3,AdIntrCity SmartBus,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,Bharat Benz A/C Sleeper (2+1),22:30,10h 15m,08:45,4.3,INR 764,26
4,Atmaram Gobus,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,VE A/C Sleeper (2+1),22:30,10h 15m,08:45,4.5,INR 900,12
...,...,...,...,...,...,...,...,...,...,...
7991,HRTC - 534,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,Ordinary 3+2 Non AC Seater,18:05,11h 15m,05:20,4.5,INR 612,34
7992,HRTC - 541,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,Ordinary 3+2 Non AC Seater,19:14,08h 56m,04:10,4.9,INR 575,37
7993,HRTC - 549,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,A/C Executive (2+3),19:25,11h 20m,06:45,3.5,INR 701,44
7994,HRTC - 557,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,Ordinary 3+2 Non AC Seater,20:15,10h 55m,07:10,2.9,INR 575,36


In [ ]:
Bus_Data['Price']

In [18]:
Bus_Data_Price= Bus_Data['Price'].str.replace('INR', '').astype(float)

In [ ]:
Bus_Data_Price

In [20]:
import mysql.connector

def connect_to_mysql():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password=""
    )

connection = connect_to_mysql()

dt = Bus_Data.fillna({
    'Bus_name': '',
    'Route_Link': '',
    'Route': '',
    'Bus_Type': '',
    'Departing_Time': '',
    'Duration': '',
    'Reaching_Time': '',
    'Star_Rating': 0,
    'Price': 0,
    'Seat_Available': 0
})

bus_names = dt['Bus_name']
route_link = dt['Route_Link']
route = dt['Route']
bus_type = dt['Bus_Type']
departing_time = dt['Departing_Time']
duration = dt['Duration']
reaching_time =dt['Reaching_Time']
star_rating = round(dt['Star_Rating'],2)
price = round(dt['Price'],2)
seat_available = dt['Seat_Available']
mycursor = connection.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS RedBus")
mycursor.execute("USE RedBus")

mycursor.execute('''CREATE TABLE IF NOT EXISTS RedBus.Bus_Data (
         Bus_name VARCHAR(200),
        Route_Link VARCHAR(200),
        Route VARCHAR(200),
        Bus_Type VARCHAR(200),
        Departing_Time TEXT,
        Duration Varchar(255),
        Reaching_Time TEXT,
        Star_Rating FLOAT,
        Price FLOAT,
        Seat_Available INT
             )''')


sql = '''
    INSERT INTO RedBus.Bus_Data
    (Bus_name, Route_Link, Route, Bus_Type, Departing_Time, Duration, Reaching_Time, Star_Rating, Price, Seat_Available) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,%s)
'''

val = zip(bus_names, route_link, route, bus_type, departing_time, duration, reaching_time, star_rating, Bus_Data_Price, seat_available)

for row in val:
    mycursor.execute(sql, row)

connection.commit() 

mycursor.close()
connection.close()

print("Data inserted successfully.")

Data inserted successfully.


In [21]:
import mysql.connector
from datetime import datetime

def connect_to_mysql():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password=""
    )

connection = connect_to_mysql()

mycursor = connection.cursor()
query = "SELECT Bus_name, Route_Link, Route, Bus_Type, Departing_Time, Duration, Reaching_Time, Star_Rating, Price, Seat_Available FROM RedBus.Bus_Data"
cursor = connection.cursor(buffered=True)
mycursor.execute(query)
data = mycursor.fetchall()
columns = ["Bus_name", "Route_Link", "Route", "Bus_Type", "Departing_Time", "Duration", "Reaching_Time", "Star_Rating", "Price", "Seat_Available"]
Red_Bus_Data = pd.DataFrame(data, columns=columns)


In [22]:
Red_Bus_Data

,Bus_name,Route_Link,Route,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Available
0,Kadamba Transport Corporation Limited (KTCL) -...,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,Volvo AC Seater 2+2,20:30,09h 51m,06:21,4.0,800.0,15
1,Atmaram Gobus,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,VE A/C Sleeper (2+1),21:00,10h 30m,07:30,4.5,799.0,17
2,Ashray Travels,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,Bharat Benz A/C Sleeper (2+1),21:00,11h 30m,08:30,4.5,849.0,20
3,AdIntrCity SmartBus,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,Bharat Benz A/C Sleeper (2+1),22:30,10h 15m,08:45,4.3,764.0,26
4,Atmaram Gobus,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,VE A/C Sleeper (2+1),22:30,10h 15m,08:45,4.5,900.0,12
...,...,...,...,...,...,...,...,...,...,...
7991,HRTC - 534,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,Ordinary 3+2 Non AC Seater,18:05,11h 15m,05:20,4.5,612.0,34
7992,HRTC - 541,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,Ordinary 3+2 Non AC Seater,19:14,08h 56m,04:10,4.9,575.0,37
7993,HRTC - 549,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,A/C Executive (2+3),19:25,11h 20m,06:45,3.5,701.0,44
7994,HRTC - 557,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,Ordinary 3+2 Non AC Seater,20:15,10h 55m,07:10,2.9,575.0,36


In [ ]:
Red_Bus_Data.count()

In [24]:
null_counts = Red_Bus_Data.isnull().sum()

In [ ]:
null_counts

In [ ]:
Red_Bus_Data['Route']

In [ ]:
Red_Bus_Data[['Departure', 'Arrival']] = Red_Bus_Data['Route'].str.split('-to-', expand=True)

print(Red_Bus_Data)

In [28]:
red_data=pd.DataFrame(Red_Bus_Data)

In [29]:
red_data

,Bus_name,Route_Link,Route,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Available,Departure,Arrival
0,Kadamba Transport Corporation Limited (KTCL) -...,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,Volvo AC Seater 2+2,20:30,09h 51m,06:21,4.0,800.0,15,pune,goa
1,Atmaram Gobus,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,VE A/C Sleeper (2+1),21:00,10h 30m,07:30,4.5,799.0,17,pune,goa
2,Ashray Travels,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,Bharat Benz A/C Sleeper (2+1),21:00,11h 30m,08:30,4.5,849.0,20,pune,goa
3,AdIntrCity SmartBus,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,Bharat Benz A/C Sleeper (2+1),22:30,10h 15m,08:45,4.3,764.0,26,pune,goa
4,Atmaram Gobus,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,VE A/C Sleeper (2+1),22:30,10h 15m,08:45,4.5,900.0,12,pune,goa
...,...,...,...,...,...,...,...,...,...,...,...,...
7991,HRTC - 534,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,Ordinary 3+2 Non AC Seater,18:05,11h 15m,05:20,4.5,612.0,34,delhi,sarkaghat
7992,HRTC - 541,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,Ordinary 3+2 Non AC Seater,19:14,08h 56m,04:10,4.9,575.0,37,delhi,sarkaghat
7993,HRTC - 549,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,A/C Executive (2+3),19:25,11h 20m,06:45,3.5,701.0,44,delhi,sarkaghat
7994,HRTC - 557,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,Ordinary 3+2 Non AC Seater,20:15,10h 55m,07:10,2.9,575.0,36,delhi,sarkaghat


In [30]:
import mysql.connector

def connect_to_mysql():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password=""
    )

connection = connect_to_mysql()

dt = red_data.fillna({
    'Bus_name': '',
    'Route_Link': '',
    'Route': '',
    'Departure':'',
    'Arrival':'',
    'Bus_Type': '',
    'Departing_Time': '',
    'Duration': '',
    'Reaching_Time': '',
    'Star_Rating': 0,
    'Price': 0,
    'Seat_Available': 0
})

bus_names = dt['Bus_name']
route_link = dt['Route_Link']
route = dt['Route']
Departure=dt['Departure']
Arrival=dt['Arrival']
bus_type = dt['Bus_Type']
departing_time = dt['Departing_Time']
duration = dt['Duration']
reaching_time =dt['Reaching_Time']
star_rating = round(dt['Star_Rating'],2)
price = round(dt['Price'],2)
seat_available = dt['Seat_Available']
mycursor = connection.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS RedBus")
mycursor.execute("USE RedBus")

mycursor.execute('''CREATE TABLE IF NOT EXISTS RedBus.Red_Bus_Data (
         Bus_name VARCHAR(200),
        Route_Link VARCHAR(200),
        Route VARCHAR(200),
        Departure TEXT,
        Arrival TEXT,
        Bus_Type VARCHAR(200),
        Departing_Time TEXT,
        Duration Varchar(255),
        Reaching_Time TEXT,
        Star_Rating FLOAT,
        Price FLOAT,
        Seat_Available INT
             )''')


sql = '''
    INSERT INTO RedBus.Red_Bus_Data
    (Bus_name, Route_Link, Route, Departure, Arrival, Bus_Type, Departing_Time, Duration, Reaching_Time, Star_Rating, Price, Seat_Available) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,%s,%s,%s)
'''

val = zip(bus_names, route_link, route, Departure, Arrival, bus_type, departing_time, duration, reaching_time, star_rating, Bus_Data_Price, seat_available)

for row in val:
    mycursor.execute(sql, row)

connection.commit() 

mycursor.close()
connection.close()

print("Data inserted successfully.")

Data inserted successfully.


In [31]:
import mysql.connector
from datetime import datetime

def connect_to_mysql():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password=""
    )

connection = connect_to_mysql()

mycursor = connection.cursor()
query1 = "SELECT Bus_name, Route_Link, Route, Departure, Arrival, Bus_Type, Departing_Time, Duration, Reaching_Time, Star_Rating, Price, Seat_Available FROM RedBus.Red_Bus_Data"
cursor = connection.cursor(buffered=True)
mycursor.execute(query1)
Values = mycursor.fetchall()
columns = ["Bus_name", "Route_Link", "Route","Departure", "Arrival", "Bus_Type", "Departing_Time", "Duration", "Reaching_Time", "Star_Rating", "Price", "Seat_Available"]
Red_Bus_Final_Data = pd.DataFrame(Values, columns=columns)

In [32]:
Red_Bus_Final_Data

,Bus_name,Route_Link,Route,Departure,Arrival,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Available
0,Kadamba Transport Corporation Limited (KTCL) -...,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,pune,goa,Volvo AC Seater 2+2,20:30,09h 51m,06:21,4.0,800.0,15
1,Atmaram Gobus,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,pune,goa,VE A/C Sleeper (2+1),21:00,10h 30m,07:30,4.5,799.0,17
2,Ashray Travels,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,pune,goa,Bharat Benz A/C Sleeper (2+1),21:00,11h 30m,08:30,4.5,849.0,20
3,AdIntrCity SmartBus,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,pune,goa,Bharat Benz A/C Sleeper (2+1),22:30,10h 15m,08:45,4.3,764.0,26
4,Atmaram Gobus,https://www.redbus.in/bus-tickets/pune-to-goa,pune-to-goa,pune,goa,VE A/C Sleeper (2+1),22:30,10h 15m,08:45,4.5,900.0,12
...,...,...,...,...,...,...,...,...,...,...,...,...
7991,HRTC - 534,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,delhi,sarkaghat,Ordinary 3+2 Non AC Seater,18:05,11h 15m,05:20,4.5,612.0,34
7992,HRTC - 541,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,delhi,sarkaghat,Ordinary 3+2 Non AC Seater,19:14,08h 56m,04:10,4.9,575.0,37
7993,HRTC - 549,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,delhi,sarkaghat,A/C Executive (2+3),19:25,11h 20m,06:45,3.5,701.0,44
7994,HRTC - 557,https://www.redbus.in/bus-tickets/delhi-to-sar...,delhi-to-sarkaghat,delhi,sarkaghat,Ordinary 3+2 Non AC Seater,20:15,10h 55m,07:10,2.9,575.0,36


In [ ]:
import mysql.connector
import pandas as pd
from datetime import datetime

def connect_to_mysql():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password=""
    )

connection = connect_to_mysql()

mycursor = connection.cursor()

query = """
SELECT DISTINCT Bus_name, Route_Link, Route, Departure, Arrival, Bus_Type, Departing_Time, Duration, Reaching_Time, Star_Rating, Price, Seat_Available 
FROM RedBus.Red_Bus_Data
"""

mycursor.execute(query)

Values = mycursor.fetchall()

columns = ["Bus_name", "Route_Link", "Route", "Departure", "Arrival", "Bus_Type", "Departing_Time", "Duration", "Reaching_Time", "Star_Rating", "Price", "Seat_Available"]

Red_Bus_Final_Data = pd.DataFrame(Values, columns=columns)

mycursor.close()
connection.close()

print(Red_Bus_Final_Data)


In [34]:
Red_Bus_Final_Data = pd.DataFrame(Values, columns=columns)

In [ ]:
Red_Bus_Final_Data.count()

In [36]:
import mysql.connector

def connect_to_mysql():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password=""
    )

connection = connect_to_mysql()

dt = Red_Bus_Final_Data.fillna({
    'Bus_name': '',
    'Route_Link': '',
    'Route': '',
    'Departure':'',
    'Arrival':'',
    'Bus_Type': '',
    'Departing_Time': '',
    'Duration': '',
    'Reaching_Time': '',
    'Star_Rating': 0,
    'Price': 0,
    'Seat_Available': 0
})

bus_names = dt['Bus_name']
route_link = dt['Route_Link']
route = dt['Route']
Departure=dt['Departure']
Arrival=dt['Arrival']
bus_type = dt['Bus_Type']
departing_time = dt['Departing_Time']
duration = dt['Duration']
reaching_time =dt['Reaching_Time']
star_rating = round(dt['Star_Rating'],2)
price = round(dt['Price'],2)
seat_available = dt['Seat_Available']
mycursor = connection.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS RedBus")
mycursor.execute("USE RedBus")

mycursor.execute('''CREATE TABLE IF NOT EXISTS RedBus.Red_Bus_Cleansed_Data (
         Bus_name VARCHAR(200),
        Route_Link VARCHAR(200),
        Route VARCHAR(200),
        Departure TEXT,
        Arrival TEXT,
        Bus_Type VARCHAR(200),
        Departing_Time TEXT,
        Duration Varchar(255),
        Reaching_Time TEXT,
        Star_Rating FLOAT,
        Price FLOAT,
        Seat_Available INT
             )''')


sql = '''
    INSERT INTO RedBus.Red_Bus_Cleansed_Data
    (Bus_name, Route_Link, Route, Departure, Arrival, Bus_Type, Departing_Time, Duration, Reaching_Time, Star_Rating, Price, Seat_Available) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,%s,%s,%s)
'''

val = zip(bus_names, route_link, route, Departure, Arrival, bus_type, departing_time, duration, reaching_time, star_rating, Bus_Data_Price, seat_available)

for row in val:
    mycursor.execute(sql, row)

connection.commit() 

mycursor.close()
connection.close()

print("Data inserted successfully.")

Data inserted successfully.
